# Bigram Language Model

From the Andrej Karpathy [Let's build GPT](https://www.youtube.com/watch?v=kCc8FmEb1nY) video.

Code for this lecture is mostly from https://github.com/karpathy/ng-video-lecture/tree/master.

This is an implementation of the Bigram Language model using pytorch, and training it over the tiny shakespere lecture.

It is a statitical model that utilizes the markov assumption to predict the next word given the previous word.

It's called bigram: meaning 2 words (previous, next)

Example:

Suppose we have a simple sentence:
“I love programming languages."

The bigrams extracted from this sentence are:

    ("I", "love")

    ("love", "programming")

    ("programming", "languages")

Using a bigram model, the probability of the sentence is computed as:
$$
P("I love programming languages")=P("I")⋅P("love"∣"I")⋅P("programming"∣"love")⋅P("languages"∣"programming")
P("I love programming languages")=P("I")⋅P("love"∣"I")⋅P("programming"∣"love")⋅P("languages"∣"programming")
$$

See more information about it here https://www.educative.io/answers/what-is-a-bigram-language-model

In [49]:
# Here we could just run the bigram.py file which contains the code from the lecture 
# This a cleaned up version of the code in this notebook.
!python3 bigram.py

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911

MARI he avayokis erceller thour d, myono thishe me tord se by he me, Forder anen: at trselorinjulour t yoru thrd wo ththathy IUShe bavidelanoby man ond be jus as g e atot Meste hrle s, ppat t JLENCOLIUS:
Oppid tes d s o ged moer y pevehear soue maramapay fo t: bueyo malalyo!
Duir.
Fl ke it I t l o'ddre d ondu s?
cr, havetrathackes w.
PUpee meshancun, hrendspouthoulouren whel's'sesoread pe, s whure our heredinsethes; sedsend r lo pamit,
QUMIVIVIOfe m ne RDINid we tr ort; t:
MINENXI l dintandore r


Let's start by building the bigram model step by step here

In [1]:
# Download the tiny shakespear dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-03 23:33:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-10-03 23:33:22 (12.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# Load the dataset to file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print("length of dataset in characters: ", len(text))
# Printing the for 1000 characters
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [9]:
# Here let's rerieve all the unique characters in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("The total number of characters in the dataset is: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
The total number of characters in the dataset is:  65


In [14]:
# Create a mapping from characters to integers
# Here using character level tokenizer

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: takes a string, outputs an integer
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world



In practice, sub-word tokenizers are used (encode a single subword into an integer).

There are other ways to encode the text into a number representation.

Google for example uses the https://github.com/google/sentencepiece

open ai for example has tiktoken https://github.com/openai/tiktoken (and that's what gpt uses)

In [16]:
# Let's now encode the entire shakespere dataset into code
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [17]:
# Let's now split the dateset into training and validation datasets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 8 # This can also be called the context window
train_data[:block_size+1]

# The block size determines the max length of the context that could be fed into the mode
# When training we will 8 training samples for each block
# That's because starting from the first word, we will feed that to the model
# and try to predict the second word. Then we could feed the first & second
# words and try to predict the third .... etc

# Here is an example of how we will feed the block during training

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [22]:
# Now will now define the batch size, batches are a bag of blocks that we will process in parallel

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions


def get_batch(split):
    # generate a small batch of data of the inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [36]:
# Let's now create the simplest language model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targes are both (batch_size or B, block_size or T) tensor of integers
        # logits below will be of size: (batch_size or B, block_size or T, vocab_size or C)
        logits = self.token_embedding_table(idx)
        loss = None
        if targets is not None:
            # Reshape things so they work with the cross entropy loss function
            B, T, C = logits.shape # B: batch_size, T: block_size, C: vocab_size
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indicies in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append the sampled index to the running sequence
            # NOTE: this step is techincally incorrect for the bigram
            # model, here we keep the entire history of the characters
            # however, the bigram model only uses the last character 
            # from the history. Hence, it is not needed to keep the entire
            # history. The reason we created this function as such is because
            # it will be used later with the transformer model.
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

# Let's try out the generation portion of the mode;
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))
print("The above generation is garbage because the model is not trained")

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ
The above generation is garbage because the model is not trained


In [42]:
# Let's now train the model

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    # sampe a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.566971778869629


In [46]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx=idx, max_new_tokens=500)[0].tolist()))


kntr I se y ce sech is arsenased tthentsman lemout:
I
We;


NGle, LAndr!
Fimau f,
QUTirourthithak, malit I myon Ispoupe mom;
H:
hades cks thaker de benlleall in
OUETCHUETHare t VABUSAmie irrowaks

Hadianse t ral-eam, are, RI ar?
CETh thosthe my, th curous ver h ch kityonoyoveeld,-w dsupu presoto-tll GLombur g!
EOREEDe, mer peanthedey ninches lom touds ag wer, g, thowlassue a'd blld he
Weenbe d: andaly mmed, Gorid, capue; ounertheyart ak:
hinod tord tty t s y,
Toroittchaceras dy, mpe g.
S:
S: ths
